# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [83]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import numpy as np
import requests
import geoviews as gv
import geoviews.feature as gf
import xarray as xr
from cartopy import crs

# Import API key
from api_keys import geoapify_key

geoapify_key = "54eb3bcc646a4addab669e000d1acbb7"

In [84]:
longitude = -118.24
latitude = 34.0522
radius = 1000


In [85]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ladang seri kundang,3.2856,101.5190,86.81,83,20,2.30,MY,1682561934
1,1,edinburgh of the seven seas,-37.0676,-12.3116,61.29,74,93,26.33,SH,1682561934
2,2,luchegorsk,46.4764,134.2569,52.41,30,100,9.98,RU,1682561934
3,3,grytviken,-54.2811,-36.5092,32.50,73,100,9.44,GS,1682561934
4,4,praya,14.9215,-23.5087,72.14,73,40,11.50,CV,1682561934


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [86]:
#%%capture --no-display

# Configure the map plot
map_plot1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidty",
    scale = 0.01,
    color = "City"
    
)

# Display the map
map_plot1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [87]:
# Narrow down cities that fit criteria and drop any results with null values
weather_df = city_data_df

weather_df = weather_df.loc[(weather_df["Max Temp"] <= 85) & (weather_df["Max Temp"] > 72)]
weather_df = weather_df.loc[weather_df["Wind Speed"] <= 5]
weather_df = weather_df.loc[weather_df["Cloudiness"] <= 30]

# Drop any rows with null values
best_weather_df = weather_df.dropna()

# Display sample data
best_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
42,42,georgetown,5.4112,100.3354,84.13,84,20,4.61,MY,1682561290
43,43,hadibu,12.6500,54.0333,81.03,75,6,2.46,YE,1682561399
98,98,cambria,35.5641,-121.0808,84.94,83,25,4.70,US,1682561408
100,100,aracruz,-19.8203,-40.2733,76.21,97,2,2.37,BR,1682561409
112,112,guaruja,-23.9931,-46.2564,72.90,26,19,4.99,BR,1682561410
114,114,west bay,19.3667,-81.4167,80.67,76,0,4.61,KY,1682561411
123,123,ghat,24.9647,10.1728,76.51,23,10,1.43,LY,1682561412
152,152,marovoay,-16.1000,46.6333,73.92,84,11,2.01,MG,1682561416
179,179,sangin,32.0728,64.8359,72.57,29,17,3.67,AF,1682561421
306,306,quepos,9.4236,-84.1652,80.91,78,15,3.60,CR,1682561438


### Step 3: Create a new DataFrame called `hotel_df`.

In [88]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = best_weather_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = " "

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
42,georgetown,MY,5.4112,100.3354,84,
43,hadibu,YE,12.6500,54.0333,75,
98,cambria,US,35.5641,-121.0808,83,
100,aracruz,BR,-19.8203,-40.2733,97,
112,guaruja,BR,-23.9931,-46.2564,26,
114,west bay,KY,19.3667,-81.4167,76,
123,ghat,LY,24.9647,10.1728,23,
152,marovoay,MG,-16.1000,46.6333,84,
179,sangin,AF,32.0728,64.8359,29,
306,quepos,CR,9.4236,-84.1652,78,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [100]:
# Set parameters to search for a hotel
radius = 10000,
params = {
    "type": "lodging",
    "key": geoapify_key,
    "limit": 20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"{lat},{lng}"
    params["bias"] = f"{lat},{lng}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
georgetown - nearest hotel: No hotel found
hadibu - nearest hotel: No hotel found
cambria - nearest hotel: No hotel found
aracruz - nearest hotel: No hotel found
guaruja - nearest hotel: No hotel found
west bay - nearest hotel: No hotel found
ghat - nearest hotel: No hotel found
marovoay - nearest hotel: No hotel found
sangin - nearest hotel: No hotel found
quepos - nearest hotel: No hotel found
newman - nearest hotel: No hotel found
nova xavantina - nearest hotel: No hotel found
la rinconada - nearest hotel: No hotel found
pahrump - nearest hotel: No hotel found
nova vicosa - nearest hotel: No hotel found
lashio - nearest hotel: No hotel found
butterworth - nearest hotel: No hotel found
riberalta - nearest hotel: No hotel found
luganville - nearest hotel: No hotel found
benjamin constant - nearest hotel: No hotel found
nicoya - nearest hotel: No hotel found
nortelandia - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
42,georgetown,MY,5.4112,100.3354,84,No hotel found
43,hadibu,YE,12.6500,54.0333,75,No hotel found
98,cambria,US,35.5641,-121.0808,83,No hotel found
100,aracruz,BR,-19.8203,-40.2733,97,No hotel found
112,guaruja,BR,-23.9931,-46.2564,26,No hotel found
114,west bay,KY,19.3667,-81.4167,76,No hotel found
123,ghat,LY,24.9647,10.1728,23,No hotel found
152,marovoay,MG,-16.1000,46.6333,84,No hotel found
179,sangin,AF,32.0728,64.8359,29,No hotel found
306,quepos,CR,9.4236,-84.1652,78,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [101]:
%%capture --no-display

# Configure the map plot
map_plot2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidty",
    scale = 0.01,
    color = "City", 
)
# Display the map
map_plot2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City)